# Reinforcement Learning, 2024-01
# Tarea 3 - Algoritmos de aprendizaje por refuerzo tabulares.

> Daniel Villar González, 201923374.  
> Daniel Alvarez, 201911320.

## **Descripción de tarea**

Para esta tarea se nos solicitó realizar modificaciones al ya conocido juego de escaleras y serpientes formulado en la entrega pasada. Para este caso se hizo uso de la implementación de algoritmos tabulares para la búsqueda de la politica óptima a tráves de un agente. Por este motivo, se aclaran las siguientes necesidades que nuestro MDP debe cumplir para este caso.

### ***Requerimientos***
1. La recompensa vista en un estado terminal es cero.
2. El siguiente estado visto por un agente en un estado terminal es igual al mismo, cumpliendo con las dinámicas del MDP.
3. Para este caso, como existe aprendizaje, no se conocen de primera mano las probabilidades de transición, por lo que el agente debe aprender.
4. Existen varios episodios donde se actualiza la matriz Q, con el fin de conocer los mayores valores del par estado-acción.

## **Librerias**

In [2]:
import numpy as np
import random

## **Punto 1**

**Considere el MDP que usted formuló para modelar este problema en el taller anterior.**
**Escriba un módulo de Python que permita la interacción de un agente con el ambiente para el MDP formulado. Su implementación debe:**
- **Dada una acción en un estado, retornar la recompensa y el estado resultante en esa acción. Considere el caso especial del estado terminal.**
- **Ejecutar una política arbitraria.**
> A continuación presentamos la contrucción del MDP establecido.

In [3]:
def construirMDP(casillasAzules, casillasRojas, p, factorDescuento):
    if factorDescuento > 1 or factorDescuento <= 0:
        return "El valor de descuento no está entre los límites permitidos"
    matrizAdelante = []
    for i in range(100):
        matrizAdelante.append([0] * 100)
    matrizRecompensasAdelante = []
    for i in range(100):
        matrizRecompensasAdelante.append([0] * 100)
    matrizAtras = []
    for i in range(100):
        matrizAtras.append([0] * 100)
    matrizRecompensasAtras= []
    for i in range(100):
        matrizRecompensasAtras.append([0] * 100)
    recompensas = {}

    for i in range(len(casillasAzules)):
        recompensas[casillasAzules[i]] = 1
    for i in range(len(casillasRojas)):
        recompensas[casillasRojas[i]] = -1

    transiciones = {8:26, 21:82, 43:77, 50:91, 54:93, 62:96, 66:87, 80:100, 98:28, 95:24, 92:51, 83:19, 73:1, 69:33, 64:36, 59:17, 55:7, 52:11, 48:9, 46:5, 44:22}
    llaves_transiciones = transiciones.keys()
    llaves_recompensas = recompensas.keys()
    for i in range(len(matrizAdelante)):
        for j in range(0, 6):
            if i + 1 not in casillasRojas and  i + 1 not in casillasAzules:
                destino = i + j + 1
                if destino >= 100:
                    destino = 99 - (destino - 100)
                if destino + 1 in llaves_transiciones:
                    matrizAdelante[i][transiciones[destino+1]-1] += p[j]
                    if destino + 1 in llaves_recompensas:
                        matrizRecompensasAdelante[i][transiciones[destino+1]-1] += recompensas[destino+1]
                else:
                    matrizAdelante[i][destino] += p[j]
                    if destino + 1 in llaves_recompensas:
                        matrizRecompensasAdelante[i][destino] += recompensas[destino+1]
    for i in range(len(matrizAtras)):
        for j in range(0, 6):
            if i + 1 not in casillasRojas and  i + 1 not in casillasAzules:
                destino = i - j
                if destino <= 0:
                    destino = abs(destino) + 1
                if destino in llaves_transiciones:
                    matrizAtras[i][transiciones[destino]-1] += p[j]
                    if destino in llaves_recompensas:
                        matrizRecompensasAtras[i][transiciones[destino]-1] += recompensas[destino]
                else:
                    matrizAtras[i][destino-1] += p[j]
                    if destino in llaves_recompensas:
                        matrizRecompensasAtras[i][destino-1] += recompensas[destino]
    return matrizAdelante, matrizAtras, matrizRecompensasAdelante, matrizRecompensasAtras

### INPUTS
casillasAzules = [80, 100]
casillasRojas = [23, 37, 45, 67, 89]
p1 = 0.1
p2 = 0.1
p3 = 0.2
p4 = 0.3
p5 = 0.2
p6 = 0.1
p = [p1, p2, p3, p4, p5, p6]
factorDescuento = 0.9
matrizAdelante, matrizAtras, matrizRecompensasAdelante, matrizRecompensasAtras = construirMDP(casillasAzules, casillasRojas, p, factorDescuento)

A continuación se presenta la matriz de recompensas al tomar la acción de ir hacia adelante. El valor en la posición (i,j) indica la recompensa obtenida al pasar del estado i y al llegar al estado j.

In [4]:
for i in range(len(matrizRecompensasAdelante)):
            print(matrizRecompensasAdelante[i])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

A continuación se presenta la matriz de recompensas al tomar la acción de ir hacia atrás.

In [5]:
for i in range(len(matrizRecompensasAtras)):
            print(matrizRecompensasAtras[i])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

A continuación se presenta la matriz de transición al tomar la acción de ir hacia adelante. El valor en la posición (i,j) indica la la probabilidad de estar en el estado i y llegar al estado j.

In [6]:
for i in range(len(matrizAdelante)):
            print(matrizAdelante[i])

[0, 0.1, 0.1, 0.2, 0.3, 0.2, 0.1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0.1, 0.1, 0.2, 0.3, 0.2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0.1, 0.1, 0.2, 0.3, 0, 0.1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0.1, 0.1, 0.2, 0, 0.2, 0.1, 0, 0, 0, 0, 0, 0, 0,

A continuación se presenta la matriz de transición al tomar la acción de ir hacia atrás.

In [7]:
for i in range(len(matrizAtras)):
            print(matrizAtras[i])

[0.1, 0.1, 0.2, 0.3, 0.2, 0.1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0.2, 0.2, 0.3, 0.2, 0.1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0.30000000000000004, 0.4, 0.2, 0.1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0.5, 0.30000000000000004, 0.2, 0, 0, 0, 0, 0, 0, 0

### ***Agente***
Para este caso se definió un agente que interactúa con el ambiente según un vector de probabilidades que le retornan su siguiente estado, ejecutando una política establecida. Para este caso se usó como prueba la ejecución de una política sencilla, la cual toma la acción de ir hacia adelante para estados menores a 50 y la acción de ir hacia atrás para estados superiores. Debido mayoritariamente a las trancisiones y al ser una política simple, no generará todo el episodio para una acción final ya que prontamente se encuentra con los valores de un estado terminal desfavorable para nuestro agente.

In [8]:
class MDPenv:
    def __init__(self, casillasAzules, casillasRojas, p, factorDescuento):
        self.matrizAdelante, self.matrizAtras, self.matrizRecompensasAdelante, self.matrizRecompensasAtras = construirMDP(casillasAzules, casillasRojas, p, factorDescuento)
        self.estado_actual = 0
        self.factorDescuento = factorDescuento

        # Obtener los valores anteriores de cada elemento en casillasAzules y casillasRojas

        # Crear lista terminal_state que contiene todos los valores anteriores de casillasAzules y casillasRojas
        self.terminal_state = casillasAzules + casillasRojas

    def accion(self, movimiento):
        """
        Toma la acción dependiendo de la decisión del agente
        sabemos que si se encuentra en estado terminal la recompensa es 0, y el
        nuevo estado es el mismo.
        """
        # for state in self.terminal_state:
        #     if self.estado_actual == state:
        #         return 0, self.estado_actual

        # Decidir la matriz de transición y de recompensas a usar basado en la acción
        if movimiento == 'adelante':
            transiciones = self.matrizAdelante[self.estado_actual]
            recompensas = self.matrizRecompensasAdelante[self.estado_actual]

        elif movimiento == 'atras':
            transiciones = self.matrizAtras[self.estado_actual]
            recompensas = self.matrizRecompensasAtras[self.estado_actual]
        # else:
        #     raise ValueError("Movimiento no válido. Use 'adelante' o 'atras'.")

        # depende de los valores de la matriz de transicion que se obtiene con p
        nuevo_estado = self._determinar_nuevo_estado(transiciones)


        # recibe recompensa
        recompensa = recompensas[nuevo_estado]
        # aquí actualizamos el estado :)
        self.estado_actual = nuevo_estado
        # devolvemos la dos cosas que ve tomada la acción recompensa y nuevo estado
        return recompensa, nuevo_estado

    # aqui se toma una decisión aleatoria de los siguientes estados en función de la matriz de transición
    # esto tiene sentido para el agente pero se debe cambiar para SARSA porque parece que no tiene aprendizaje
    def _determinar_nuevo_estado(self, transiciones):
        return random.choices(range(100), weights=transiciones)[0]


    def ejecutar_politica(self, politica, pasos):
        """
        Ejecuta una política dada durante un número especificado de pasos.
        política es una función que toma un estado y devuelve una acción.
        """
        historia = []
        termino = False
        for _ in range(pasos):
            if termino == False:
                estadoAnterior = self.estado_actual
                accion = politica(self.estado_actual)
                recompensa, self.estado_actual = self.accion(accion)
                movimiento = self.estado_actual - estadoAnterior
                historia.append((estadoAnterior + 1 , accion, recompensa, self.estado_actual + 1, movimiento))
                if self.estado_actual + 1 in self.terminal_state:
                    termino = True
        return historia

    def ejecutar_politica_T(self, politica):
        """
        Ejecuta una política dada durante hasta un estado terminal
        """
        historia = []
        termino = False
        while not termino:
            estadoAnterior = self.estado_actual
            accion = politica(self.estado_actual)
            recompensa, self.estado_actual = self.accion(accion)
            movimiento = self.estado_actual - estadoAnterior
            historia.append((estadoAnterior + 1 , accion, recompensa, self.estado_actual + 1, movimiento))
            if self.estado_actual + 1 in self.terminal_state:
                termino = True
        return historia

    def ejecutar_politica_Q(self, politica):
        """
        Ejecuta una política dada durante hasta un estado terminal para un Q dado
        """
        historia = []
        termino = False
        while not termino:
            estadoAnterior = self.estado_actual
            accion = politica[self.estado_actual]
            recompensa, self.estado_actual = self.accion(accion)
            movimiento = self.estado_actual - estadoAnterior
            historia.append((estadoAnterior + 1 , accion, recompensa, self.estado_actual + 1, movimiento))
            if self.estado_actual + 1 in self.terminal_state:
                termino = True
        return historia

    def reset_state(self, estado_i):
        self.estado_actual = estado_i


A continuación mostramos un episodio con todos los estados iniciales, acciones tomadas, recompensas recibidas, estados finales y el movimiento realizado. Como se puede notar, el último estado es un estado terminal (en este caso con recompensa -1).

In [9]:
# TEST

def politica_simple(estado):
    return 'adelante' if estado < 50 else 'atras'

mdp = MDPenv([80, 100], [23, 37, 45, 67, 89], [0.1, 0.1, 0.2, 0.3, 0.2, 0.1], 0.99)
historia = mdp.ejecutar_politica_T(politica_simple)
print(historia)

[(1, 'adelante', 0, 2, 1), (2, 'adelante', 0, 6, 4), (6, 'adelante', 0, 26, 20), (26, 'adelante', 0, 31, 5), (31, 'adelante', 0, 35, 4), (35, 'adelante', 0, 39, 4), (39, 'adelante', -1, 45, 6)]


## **Punto 2**

**Usando su módulo del MDP, implemente el algoritmo de control de Montecarlo off-policy para encontrar una política óptima.**

A continuación se presenta el algoritmo y se imprimen todos los episodios con las siguiente información: Recompensa obtenida al finalizar el episodio, estado terminal y número del episodio.

In [30]:
class MonteCarloOffPolicyAgentQ:
    def __init__(self, num_states, discount_factor=0.99):
        self.num_states = num_states
        self.discount_factor = discount_factor
        self.pi = np.zeros(num_states)
        self.Q = np.zeros((num_states, 2))  # Inicialización la matriz Q
        self.C = np.zeros((num_states, 2))  # Suma de los pesos de muestreo por importancia acumulados para cada estado-accion

    def choose_action(self, state, behavior_policy):
        return behavior_policy(state)

    def update_Q(self, episode, target_policy):
        G = 0
        W = 1
        for state, action_index, reward in reversed(episode):
            G = reward + self.discount_factor * G
            self.C[state, action_index] += W
            # esta parate va hasta C = C + W
            self.Q[state, action_index] += (W / self.C[state, action_index]) * (G - self.Q[state, action_index])
            self.pi[state] = np.argmax(self.Q[state, action_index])
            if action_index != target_policy(state):
                break  # Si la acción tomada no es la misma que habría tomado la política objetivo, termina el loop
            W *= 1.0 / 0.5  # Actualizar W por el cociente de las probabilidades bajo las políticas objetivo y de comportamiento

    def get_policy(self):
        # Obtener la política basada en las estimaciones de Q (la acción con el valor Q más alto para cada estado)
        policy_text = []
        for state in range(self.num_states):
            action_index = np.argmax(self.Q[state])
            action = 'adelante' if action_index == 0 else 'atras'
            policy_text.append(action)
        return policy_text


def behavior_policy(state):
    return 'adelante' if random.random() < 0.5 else 'atras'  # Política completamente aleatoria

def target_policy(state):
    # Política greedy basada en las estimaciones actuales de la función de valor, asumiendo dos acciones
    return 'adelante' if np.random.rand() < 0.7 else 'atras'  # Aquí, puedes hacerlo dependiente de V

def generate_episode(mdp, agent, behavior_policy):
    mdp.reset_state(0)  # Estado inicial
    episode = []
    terminal = False
    while not terminal:
        state = mdp.estado_actual
        action = agent.choose_action(state, behavior_policy)
        action_index = 1 if action == "adelante" else 0
        reward, next_state = mdp.accion(action)
        episode.append((state, action_index, reward))
        if next_state + 1 in mdp.terminal_state:
            terminal = True
            print("Recompensa: " + str(reward))
            print("Estado terminal: " + str(next_state + 1))
    return episode

def train_monte_carlo_off_policy(mdp, agent, num_episodes, behavior_policy, target_policy):
    for i in range(num_episodes):
        episode = generate_episode(mdp, agent, behavior_policy)
        print("Episodio: " + str(i+1))
        print("___________________")
        agent.update_Q(episode, target_policy)



# Instanciación y entrenamiento
num_states = 100
num_actions = 2  # 'adelante' o 'atras'
mdp = MDPenv([80, 100], [23, 37, 45, 67, 89], [0.1, 0.1, 0.2, 0.3, 0.2, 0.1], 0.99)
mc_off_policy_agent = MonteCarloOffPolicyAgentQ(num_states, num_actions)
train_monte_carlo_off_policy(mdp, mc_off_policy_agent, 100000, behavior_policy, target_policy)
optimal_policy_mc_text  = mc_off_policy_agent.get_policy()



Se truncaron las últimas líneas 5000 del resultado de transmisión.
Recompensa: -1
Estado terminal: 23
Episodio: 98751
___________________
Recompensa: -1
Estado terminal: 45
Episodio: 98752
___________________
Recompensa: -1
Estado terminal: 23
Episodio: 98753
___________________
Recompensa: -1
Estado terminal: 23
Episodio: 98754
___________________
Recompensa: -1
Estado terminal: 37
Episodio: 98755
___________________
Recompensa: -1
Estado terminal: 23
Episodio: 98756
___________________
Recompensa: -1
Estado terminal: 37
Episodio: 98757
___________________
Recompensa: -1
Estado terminal: 23
Episodio: 98758
___________________
Recompensa: -1
Estado terminal: 67
Episodio: 98759
___________________
Recompensa: -1
Estado terminal: 23
Episodio: 98760
___________________
Recompensa: -1
Estado terminal: 23
Episodio: 98761
___________________
Recompensa: -1
Estado terminal: 45
Episodio: 98762
___________________
Recompensa: -1
Estado terminal: 23
Episodio: 98763
___________________
Recompensa

A continuación se presenta la política encontrada:

In [35]:
print(optimal_policy_mc_text)

['adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'atras', 'atras', 'atras', 'atras', 'atras', 'atras', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'atras', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'atras', 'adelante', 'atras', 'adelante', 'atras', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'atras', 'adelante', 'atras', 'atras', 'atras', 'adelante', 'atras', 'atras', 'atras', 'atras', 'atras', 'atras', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'ad

A continuación se presenta la función winning_rate(sessions, policy) que calcula varias métricas relacionadas con el desempeño de una política representado por un número dado de sesiones (sessions). Esta función retorna una tupla que contiene:

historical: Lista de resultados históricos de cada sesión (-1 para pérdida, 0 para empate, 1 para victoria).

winning: Proporción de sesiones ganadas respecto al total de sesiones.

step_all: Promedio de pasos por sesión en todas las sesiones.

steps_winning: Promedio de pasos por sesión en sesiones ganadas.

steps_loss: Promedio de pasos por sesión en sesiones perdidas.

In [38]:
def winning_rate(sessions, policy):
    historical = [0] * sessions
    winning = 0.0
    loss = 0.0
    steps_winning = 0
    steps_loss = 0
    step_all = 0.0
    for i in range(sessions):
        mdp = MDPenv([80, 100], [23, 37, 45, 67, 89], [0.1, 0.1, 0.2, 0.3, 0.2, 0.1], 0.99)
        session = mdp.ejecutar_politica_Q(policy)
        step_all += len(session)
        if session[-1][2] > 0:
            historical[i] = 1
            winning += 1
            steps_winning += len(session)
        else:
            historical[i] = -1
            loss += 1
            steps_loss += int(len(session))
    steps_loss = steps_loss / loss
    step_all = step_all / sessions
    steps_winning = steps_winning / (winning+0.000001)
    winning = winning / sessions
    return historical, winning, step_all, steps_winning, steps_loss

In [39]:
history3, rate3, s_all3, s_winning3, s_loss3  = winning_rate(1000, optimal_policy_mc_text)
print("La tasa de victoria de la política es: " + str(rate3*100)+"%")
print("Los pasos promedio de las partidas son: " + str(s_all3))
print("Los pasos promedio de las partidas ganadas son: " + str(s_winning3))
print("Los pasos promedio de las partidas perdidas son: " + str(s_loss3))

La tasa de victoria de la política es: 0.0%
Los pasos promedio de las partidas son: 11.499
Los pasos promedio de las partidas ganadas son: 0.0
Los pasos promedio de las partidas perdidas son: 11.499


Esta clase representa un agente que utiliza el algoritmo de Montecarlo off-policy para aprender y actualizar los valores de la función de valor de acción Q(s,a).

El método choose_action(self, state, behavior_policy) selecciona una acción para un estado dado utilizando una política de comportamiento (behavior_policy).
La política de comportamiento puede ser aleatoria (behavior_policy) o determinada por alguna regla específica.

El método update_Q(self, episode, target_policy) actualiza la matriz de Q-values Q basándose en un episodio generado. El argumento episode es una lista que contiene tuplas (state, action_index, reward) para cada paso del episodio, y el argumento target_policy es la política objetivo que guía las actualizaciones de Q-values. Este método recorre el episodio en orden inverso (desde el último paso hasta el primero), calcula el retorno G acumulado hacia atrás para cada paso del episodio, actualiza la matriz C sumando el peso de muestreo por importancia W para el par estado-acción y actualiza el valor Q(s,a) utilizando la regla de actualización de Montecarlo off-policy: Q(s,a)←Q(s,a)+ W/C(s,a)*(G-Q(s,a)). Por último, actualiza la política π del estado s seleccionando la acción que maximiza el valor de Q(s,a).

Por otro lado, el método behavior_policy(state) define la política de comportamiento del agente, que puede ser aleatoria. El método target_policy(state) define la política objetivo del agente, que puede ser determinada por valores actuales de Q-values, y el método generate_episode(mdp, agent, behavior_policy) genera un episodio utilizando la política de comportamiento y el entorno definido por mdp.

Finalmente, el método train_monte_carlo_off_policy se encarga de entrenar el agente utilizando el algoritmo de Montecarlo off-policy durante un número especificado de episodios. Esta función genera episodios utilizando generate_episode, y luego actualiza los Q-values del agente utilizando agent.update_Q.

Este algoritmo permite que el agente actualice sus estimaciones de valor de acción utilizando episodios generados por una política de comportamiento, mientras sigue una política objetivo para las actualizaciones. Esto permite al agente aprender una política óptima incluso cuando la política de comportamiento y la política objetivo son diferentes.

## **Punto 3**

**Usando su módulo del MDP, implemente el algoritmo SARSA on-policy para encontrar una política óptima.**

SARSA (estado-acción-recompensa-estado-acción) es un algoritmo de diferencias temporales "on-policy" que nos permite mover el valor de la matriz $Q(S,A)$ hacia un valor conocido como el estimativo real de la función. Para este caso $[R + \lambda Q(S', A')]$, en particular a tráves de la expresión vista como:
$$
Q (S, A) \leftarrow Q(S, A) + \alpha[R + \lambda Q(S', A) - Q(S,A)]
$$
Nótese que para valores de $\alpha$ (tasa de aprendizaje) cercanos a 1 el estimativo se mueve hacia el estimativo real de la expresión estableciendo un reemplazo sin consideración de las iteraciónes anteriores, sin embargo, esto no sería provechoso para una matriz con valores diferentes de 0.
Para este caso, cada acción se escoje con $Q(\epsilon - greedy)$ por lo que inicialmente nos interesará que la tasa de exploración tenga un valor alto para no quedarnos en la explotación de los primeros valores mientras este decrece asimptóticamente a lo largo del episodio, en particular para los que no se encuentral con un S terminal que pueden ser varios. A continuación presentamos el siguiente macroalgoritmo de SARSA:

1. **Inicialización:**
   - Inicializar la matriz de valores Q con valores arbitrarios o a cero.
   - Establecer los parámetros de control, como la tasa de aprendizaje $\alpha$, el factor de descuento $\lambda$, y la tasa de exploración inicial $\epsilon$.

2. **Para cada episodio:**
   - Reiniciar el entorno al estado inicial $S$.
   - Elegir la acción $A$ usando una política $\epsilon$-greedy basada en los valores actuales de $Q$ para $S$.
   - Mientras el episodio no haya terminado:
     - Tomar la acción $A$ y observar la recompensa $R$ y el próximo estado $S'$.
     - Elegir la próxima acción $A'$ usando una política $\epsilon$-greedy basada en los valores actuales de $Q$ para $S'$.
     - Actualizar el valor de $Q(S, A)$ utilizando la regla de actualización SARSA:
       $$
       Q(S, A) \leftarrow Q(S, A) + \alpha [R + \lambda Q(S', A') - Q(S, A)]
       $$
     - Establecer $S \leftarrow S'$ y $A \leftarrow A'$.

3. **Hasta que se cumpla el criterio de terminación.**

4. **Fin.**

A continuación se presenta el algoritmo y se imprimen todos los episodios con las siguiente información: Número del episodio, recompensa obtenida al finalizar el episodio y estado terminal.

In [11]:
class SarsaAgent:
    def __init__(self, num_states, discount_factor=0.7, alpha=0.6, epsilon=0.1):
        self.q_table = np.zeros((num_states, 2))  # acciones
        self.discount_factor = discount_factor
        self.alpha = alpha
        self.epsilon = epsilon
        self.epsilon_init = epsilon

    def choose_action(self, state):
        if np.random.rand() < self.epsilon:
            return 'adelante' if np.random.rand() < 0.5 else 'atras'  # Acción aleatoria
        else:
            action_index = np.argmax(self.q_table[state])
            return 'adelante' if action_index == 0 else 'atras'  # Mejor acción conocida

    def update_q_table(self, state, action, reward, next_state, next_action):
        action_index = 0 if action == 'adelante' else 1
        next_action_index = 0 if next_action == 'adelante' else 1
        predict = self.q_table[state, action_index]
        target = reward + self.discount_factor * self.q_table[next_state, next_action_index]
        self.q_table[state, action_index] += self.alpha * (target - predict)

    def reset_epsilon(self):
        self.epsilon = self.epsilon_init

def train_sarsa(mdp, agent, num_episodes):
    for episode in range(num_episodes):
        mdp.reset_state(0)
        state = mdp.estado_actual
        action = agent.choose_action(state)
        terminal = False
        print("___________________")
        print("episodio: " + str(episode))
        counter = 0 # epsilon asimptotico
        while not terminal:
            reward, next_state = mdp.accion(action)
            next_action = agent.choose_action(next_state)
            agent.update_q_table(state, action, reward, next_state, next_action)
            state = next_state
            action = next_action
            if state + 1 in mdp.terminal_state:
                terminal = True
                print(reward)
                print("Estado terminal: " + str(state + 1))

def get_policy(q_table):
    commands = [None] * len(q_table)
    for i in range(len(q_table)):
        if q_table[i, 0] > q_table[i, 1]:
            commands[i] = 'adelante'
        else:
            commands[i] = 'atras'
    return commands

# Instanciación y entrenamiento
mdp = MDPenv([80, 100], [23, 37, 45, 67, 89], [0.1, 0.1, 0.2, 0.3, 0.2, 0.1], 0.99)
num_states = 100
sarsa_agent = SarsaAgent(num_states)
print(mdp.terminal_state)
train_sarsa(mdp, sarsa_agent, num_episodes=100000)
policy = get_policy(sarsa_agent.q_table)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
___________________
episodio: 98750
1
Estado terminal: 100
___________________
episodio: 98751
1
Estado terminal: 100
___________________
episodio: 98752
1
Estado terminal: 100
___________________
episodio: 98753
1
Estado terminal: 100
___________________
episodio: 98754
-1
Estado terminal: 37
___________________
episodio: 98755
-1
Estado terminal: 37
___________________
episodio: 98756
-1
Estado terminal: 37
___________________
episodio: 98757
-1
Estado terminal: 37
___________________
episodio: 98758
-1
Estado terminal: 23
___________________
episodio: 98759
-1
Estado terminal: 23
___________________
episodio: 98760
-1
Estado terminal: 45
___________________
episodio: 98761
1
Estado terminal: 100
___________________
episodio: 98762
-1
Estado terminal: 37
___________________
episodio: 98763
-1
Estado terminal: 23
___________________
episodio: 98764
-1
Estado terminal: 23
___________________
episodio: 98765
-1
Estado te

A continuación se imprime la política obtenida.

In [13]:
print(policy)

['adelante', 'atras', 'atras', 'adelante', 'adelante', 'atras', 'adelante', 'atras', 'adelante', 'adelante', 'adelante', 'atras', 'adelante', 'atras', 'adelante', 'atras', 'adelante', 'atras', 'atras', 'atras', 'atras', 'atras', 'atras', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'adelante', 'atras', 'atras', 'atras', 'adelante', 'atras', 'adelante', 'atras', 'adelante', 'atras', 'atras', 'adelante', 'adelante', 'atras', 'atras', 'atras', 'atras', 'adelante', 'atras', 'adelante', 'atras', 'adelante', 'atras', 'atras', 'atras', 'atras', 'adelante', 'adelante', 'adelante', 'atras', 'atras', 'atras', 'atras', 'atras', 'atras', 'atras', 'atras', 'atras', 'adelante', 'atras', 'adelante', 'adelante', 'adelante', 'atras', 'adelante', 'adelante', 'adelante', 'adelante', 'atras', 'atras', 'atras', 'adelante', 'atras', 'atras', 'atras', 'atras', 'atras', 'atras', 'atras', 'atras', 'atras', 'adelante', 'atras', 'adelante', 'adelante', 'atras', 'adelante', 'adelante', 

In [40]:
history, rate, s_all, s_winning, s_loss  = winning_rate(1000, policy)
print("La tasa de victoria de la política es: " + str(rate*100)+"%")
print("Los pasos promedio de las partidas son: " + str(s_all))
print("Los pasos promedio de las partidas ganadas son: " + str(s_winning))
print("Los pasos promedio de las partidas perdidas son: " + str(s_loss))

La tasa de victoria de la política es: 40.699999999999996%
Los pasos promedio de las partidas son: 41.082
Los pasos promedio de las partidas ganadas son: 40.86977876936172
Los pasos promedio de las partidas perdidas son: 41.227655986509276


Como se puede notar, los pasos promedio de las partidas ganadas tienen un valor mayor que los pasos promedio de las partidas perdidas. Esto tiene sentido, pues las casillas de victoria solo se encuentran al final del tablero, mientras que las de derrota están más hacia el inicio. Se obtiene una tasa de victoria de 40% lo cual no es realmente bueno.

## **Punto 4**

Para el caso de Q-learning, es un algoritmo de diferencias temporales "off-policy" que nos permite mover el valor de la matriz $Q(S,A)$ hacia un valor conocido como el estimativo real de la función, en este caso $[R + \lambda\max_a(Q(S', a))]$, a través de la expresión vista como:

$$
Q (S, A) \leftarrow Q(S, A) + \alpha[R + \lambda\max_aQ(S', a) - Q(S,A)]
$$

En donde para cada actualización de la tabla hacemos uso de otra política que se trata de la máxima de $Q(S', a)$ sobre acciones, esta no siendo epsilon greedy, esta se usa con el fin de que la politica de selección de acción tenga una tasa de exploración, para este caso conocida como $\epsilon$
**Q-learning:**

1. **Inicialización:**
   - Inicializar la matriz de valores Q con valores arbitrarios o a cero.
   - Establecer los parámetros de control, como la tasa de aprendizaje $\alpha$, el factor de descuento $\lambda$, y la tasa de exploración inicial $\epsilon$.

2. **Para cada episodio:**
   - Reiniciar el entorno al estado inicial $S$.
   - Elegir la acción $A$ usando una política $\epsilon$-greedy basada en los valores actuales de $Q$ para $S$.
   - Mientras el episodio no haya terminado:
     - Tomar la acción $A$ y observar la recompensa $R$ y el próximo estado $S'$.
     - Elegir la próxima acción $A'$ usando una política $\epsilon$-greedy basada en los valores actuales de $Q$ para $S'$.
     - Actualizar el valor de $Q(S, A)$ utilizando la regla de actualización Q-learning:
       $$
       Q(S, A) \leftarrow Q(S, A) + \alpha [R + \lambda \max_a Q(S', a) - Q(S, A)]
       $$
     - Establecer $S \leftarrow S'$ y $A \leftarrow A'$.

3. **Hasta que se cumpla el criterio de terminación.**

4. **Fin.**

In [15]:
class QLearningAgent:
    def __init__(self, num_states, discount_factor=0.99, alpha=0.5, epsilon=0.1):
        self.q_table = np.zeros((num_states, 2))  # 2 acciones
        self.discount_factor = discount_factor
        self.alpha = alpha
        self.epsilon = epsilon

    def choose_action(self, state):
        if np.random.rand() < self.epsilon:
            return 'adelante' if np.random.rand() < 0.5 else 'atras'  # Acción aleatoria
        else:
            action_index = np.argmax(self.q_table[state])
            return 'adelante' if action_index == 0 else 'atras'  # Mejor acción conocida

    def update_q_table(self, state, action, reward, next_state):
        action_index = 0 if action == 'adelante' else 1
        predict = self.q_table[state, action_index]
        target = reward + self.discount_factor * np.max(self.q_table[next_state])
        self.q_table[state, action_index] += self.alpha * (target - predict)

def train_q_learning(mdp, agent, num_episodes):
    for episode in range(num_episodes):
        mdp.reset_state(0)
        state = mdp.estado_actual
        terminal = False
        print("___________________")
        print("episodio: " + str(episode))
        while not terminal:
            action = agent.choose_action(state)
            reward, next_state = mdp.accion(action)
            agent.update_q_table(state, action, reward, next_state)
            state = next_state
            if state + 1 in mdp.terminal_state:
                terminal = True
                print("Estado terminal: " + str(state + 1))

# Instanciación y entrenamiento
mdp = MDPenv([80, 100], [23, 37, 45, 67, 89], [0.1, 0.1, 0.2, 0.3, 0.2, 0.1], 0.99)
num_states = 100
ql_agent = QLearningAgent(num_states)
train_q_learning(mdp, ql_agent, num_episodes=10000)
policy_q = get_policy(ql_agent.q_table)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
episodio: 8333
Estado terminal: 100
___________________
episodio: 8334
Estado terminal: 100
___________________
episodio: 8335
Estado terminal: 37
___________________
episodio: 8336
Estado terminal: 100
___________________
episodio: 8337
Estado terminal: 100
___________________
episodio: 8338
Estado terminal: 37
___________________
episodio: 8339
Estado terminal: 23
___________________
episodio: 8340
Estado terminal: 100
___________________
episodio: 8341
Estado terminal: 23
___________________
episodio: 8342
Estado terminal: 100
___________________
episodio: 8343
Estado terminal: 37
___________________
episodio: 8344
Estado terminal: 100
___________________
episodio: 8345
Estado terminal: 100
___________________
episodio: 8346
Estado terminal: 23
___________________
episodio: 8347
Estado terminal: 100
___________________
episodio: 8348
Estado terminal: 100
___________________
episodio: 8349
Estado terminal: 100
_______

In [16]:
history2, rate2, s_all2, s_winning2, s_loss2  = winning_rate(1000, policy_q)
print("La tasa de victoria de la política es: " + str(rate2*100)+"%")
print("Los pasos promedio de las partidas son: " + str(s_all2))
print("Los pasos promedio de las partidas ganadas son: " + str(s_winning2))
print("Los pasos promedio de las partidas perdidas son: " + str(s_loss2))

La tasa de victoria de la política es: 94.69999999999999%
Los pasos promedio de las partidas son: 138.709
Los pasos promedio de las partidas ganadas son: 141.28511087645197
Los pasos promedio de las partidas perdidas son: 92.67924528301887


Como se puede notar, los pasos promedio de las partidas ganadas tienen un valor mayor que los pasos promedio de las partidas perdidas. Esto tiene sentido, pues las casillas de victoria solo se encuentran al final del tablero, mientras que las de derrota están más hacia el inicio. Se obtiene una tasa de victoria de 95% lo cual es considerado bastante positivo.

## **Punto 5**

**Ejecute 1000 partidas con cada una de las políticas encontradas y compare los resultados obtenidos en cuanto a:**

- **Porcentaje de partidas ganadas.**
- **Número de pasos promedio en los episodios.**

A continuación se presentan los resultados obtenidos.

### Montecarlo off-policy

In [57]:
history3, rate3, s_all3, s_winning3, s_loss3  = winning_rate(1000, optimal_policy_mc_text)
print("La tasa de victoria de la política es: " + str(rate3*100)+"%")
print("Los pasos promedio de las partidas son: " + str(s_all3))
print("Los pasos promedio de las partidas ganadas son: " + str(s_winning3))
print("Los pasos promedio de las partidas perdidas son: " + str(s_loss3))

La tasa de victoria de la política es: 0.0%
Los pasos promedio de las partidas son: 11.117
Los pasos promedio de las partidas ganadas son: 0.0
Los pasos promedio de las partidas perdidas son: 11.117


### SARSA

In [46]:
history, rate, s_all, s_winning, s_loss  = winning_rate(1000, policy)
print("La tasa de victoria de la política es: " + str(rate*100)+"%")
print("Los pasos promedio de las partidas son: " + str(s_all))
print("Los pasos promedio de las partidas ganadas son: " + str(s_winning))
print("Los pasos promedio de las partidas perdidas son: " + str(s_loss))

La tasa de victoria de la política es: 40.0%
Los pasos promedio de las partidas son: 42.194
Los pasos promedio de las partidas ganadas son: 44.4149998889625
Los pasos promedio de las partidas perdidas son: 40.71333333333333


### Q-Learning

In [50]:
history2, rate2, s_all2, s_winning2, s_loss2  = winning_rate(1000, policy_q)
print("La tasa de victoria de la política es: " + str(rate2*100)+"%")
print("Los pasos promedio de las partidas son: " + str(s_all2))
print("Los pasos promedio de las partidas ganadas son: " + str(s_winning2))
print("Los pasos promedio de las partidas perdidas son: " + str(s_loss2))

La tasa de victoria de la política es: 95.19999999999999%
Los pasos promedio de las partidas son: 139.851
Los pasos promedio de las partidas ganadas son: 143.6165964877977
Los pasos promedio de las partidas perdidas son: 65.16666666666667


Como se puede notar, el mejor resultado fue el de Q-Learning, ya que se obtiene un porcentaje de partidas ganadas cercano al 95%. A su vez, se puede notar que el número de pasos promedio fue el más alto, con un valor cercano a 140 pasos en los episodios.

Esto puede deberse a las siguientes razones:

- Q-Learning es un método off-policy, lo que significa que aprende una política óptima sin necesidad de seguir una política específica durante el entrenamiento. Esto puede llevar a una mejor convergencia hacia una política óptima en comparación con SARSA, que es un método on-policy y sigue la política actual durante el aprendizaje.

- Q-Learning utiliza una regla de actualización basada en el máximo valor Q del siguiente estado, lo que puede ser más robusto y conducir a una mejor estimación de los valores Q óptimos en comparación con SARSA, que actualiza los valores Q en función de la acción seleccionada bajo la política actual.

- Q-Learning tiende a ser más estable y puede converger hacia una política óptima de manera más consistente en entornos complejos debido a su naturaleza off-policy y su capacidad para aprender de experiencias futuras.

En todos los casos se utilizó el mismo dado del taller pasado, de manera que puediéramos comparar los resultados con la política óptima obtenida en ese taller.

En el taller anterior se concluyó que para la iteración de política se evidenció una tasa de victoria del 99.5%, mientras que para la iteración de valor se evidenció una tasa de victoria del 96%. Como se puede notar, esos resultados son superiores a los resultados de este taller. Lo anterior sucede porque en este taller no se conocen las probabilidades de transición desde un inicio, como sí sucedía en el taller anterior. Eso quiere decir que en este taller el agente debía aprender, por lo que era más difícil llegar a una política que ganara la mayor cantidad de veces.